##0.Librerías necesarias

In [12]:
#!pip install rarfile
#!pip install unzip
!pip install stop-words
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
#Librerias necesarias
import numpy as np
import pandas as pd
from google.colab import drive
import random
#import rarfile # para descomprimir archivos
import json
#import csv 
#para limpiar los datos
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('omw-1.4')
import string
import unicodedata
from num2words import num2words
#para reducir la dimensión
from sklearn.manifold import TSNE
#para representar
import matplotlib.pyplot as plt

#análisis exloratorio
from collections import Counter
from nltk import ngrams
from nltk.probability import FreqDist
from wordcloud import WordCloud

#preprocesado
from stop_words import get_stop_words
import unicodedata
import re

#modelado
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#reporte
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##1. Carga de datos

In [16]:
# Montamos GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#cargamos los datos necesarios desde nuestro google drive
path = '/content/drive/MyDrive/NLP/'

train_clean = pd.read_csv(f'{path}train_clean.csv')
test_clean= pd.read_csv(f'{path}test_clean.csv')

In [18]:
train_clean

,review_clean,sentiment_label
0,pie arrived wa missing one ordered topping pay...,1
1,burger wa fabulous bacon tasted like one green...,0
2,food wa good chicken spam meat,1
3,solid taco place tortilla bit soft,1
4,mahi mahi taco pretty good landlocked tucson c...,1
...,...,...
4760,ok burger price fry usually dont like kind dry...,1
4761,ordered cheesy todd cheeseburger ha bacon jala...,1
4762,coffee surprisingly good omeletes well made an...,1
4763,secret theyve got best burger around,0


In [19]:
test_clean

,review_clean,sentiment_label
0,next time want order bibim naengmyeon dumpling...,0
1,love place exelent steak chumichurri steak per...,0
2,server suggested salt vinegar seasoning tater ...,1
3,green bean shriveled quite honestly worst gree...,1
4,one put sign saying chicken fried nationwell k...,1
...,...,...
1584,love seafood place sits right water front lot ...,0
1585,starter fritto miso fried calamari zucchini sh...,0
1586,aw ha always smooth root beer forgotten sweet ...,0
1587,tilapia crispy taste okay onion ring little bl...,1


## Split de los datos

In [20]:
#dividimos con las columnas 'processedReview' y las etiquetas 'sentiment_label'

#split conjunto de train
X_train = train_clean['review_clean']
y_train = train_clean['sentiment_label']

#split conjunto de test
X_test = train_clean['review_clean']
y_test = train_clean['sentiment_label']


In [21]:
X_train.iloc[:10]

0    pie arrived wa missing one ordered topping pay...
1    burger wa fabulous bacon tasted like one green...
2                       food wa good chicken spam meat
3                   solid taco place tortilla bit soft
4    mahi mahi taco pretty good landlocked tucson c...
5    photograph speak volume taste delivered smoke ...
6          fry die fried cod sandwich wa best ive ever
7     crab soup delicious sandwich poboys taco awesome
8    people working need smile music yes place need...
9    ive tried variety sushi roll good yama salad w...
Name: review_clean, dtype: object

In [22]:
y_train.iloc[:10]

0    1
1    0
2    1
3    1
4    1
5    0
6    0
7    0
8    1
9    0
Name: sentiment_label, dtype: int64

## Extracción de características

In [26]:
class FeatureExtraction():
  def __init__(self, max_df: float=0.95, min_df: int=3, max_features: int =2500, ngram_range: tuple=(1, 3)):
    """Instanciamos dentro de la propia clase las clases TfidfVectorizer, CountVectorizer y SelectKBest"""
    self.tdidf = TfidfVectorizer(
                max_df=max_df,
                min_df=min_df,
                max_features=max_features,
                strip_accents='ascii',
                ngram_range=ngram_range
            )
    self.vectorizer = CountVectorizer()
    self.selector = selector = SelectKBest(chi2)
  
  def run(self,X_train: pd.Series, X_test: pd.Series, y_train: pd.Series, y_test: pd.Series) -> dict:
    #Realizamos el tdidf
    X_train_tdif, X_test_tdif = self.tdidfTraining(X_train=X_train, X_test = X_test)
    #Realizamos el count vectorizer
    X_train__vec, X_test_vec = self.countVectorizerTraining(X_train=X_train, X_test = X_test)

    #Realizamos la seleccion chi2
    X_train_tdif_best, X_test_tdif_best = self.selectKBestChi(X_train = X_train_tdif, X_test= X_test_tdif, y_train= y_train, y_test= y_test)
    X_train_vec_best, X_test_vec_best = self.selectKBestChi(X_train = X_train__vec, X_test= X_test_vec, y_train= y_train, y_test= y_test)
    #Almacenamos todos los sets en un diccionario
    dic_best_set = {'X_train_tdif_best': X_train_tdif_best, 'X_train_vec_best': X_train_vec_best, 'X_test_tdif_best': X_test_tdif_best, 'X_test_vec_best': X_test_vec_best}
    return dic_best_set

  def tdidfTraining(self, X_train: pd.Series, X_test: pd.Series) -> tuple:
    """funcion que  realiza el entrenamiento de tdidf y transforma el set de train y test
    : param X_train: pd.Series con los datos de train
    : param X_test: pd.Series con los datos de test
    : return: tupla con las series transformadas X_train_tdif, X_test_tdif"""
    #Entrenamos el modelo a través de la instancia self.tdidf
    print(f'[INFO] Realizando entrenamiento {type(self.tdidf).__name__}...')
    self.tdidf.fit(X_train)
    #Obtenemos los set de datos modificados con tdif
    print(f'[INFO] Realizando transformacion de tipo {type(self.tdidf).__name__}...')
    X_train_tdif = self.tdidf.transform(X_train)
    X_test_tdif = self.tdidf.transform(X_test)
    return X_train_tdif, X_test_tdif

  def countVectorizerTraining(self, X_train: pd.Series, X_test: pd.Series) -> tuple:
    """funcion que  realiza el entrenamiento de countVectorizer y transforma el set de train y test
    : param X_train: pd.Series con los datos de train
    : param X_test: pd.Series con los datos de test
    : return: tupla con las series transformadas X_train_vec, X_test_vec"""
    #Realizamos el entrenamiento y la transformación de train
    print(f'[INFO] Realizando entrenamiento {type(self.vectorizer).__name__}...')
    X_train_vec = self.vectorizer.fit_transform(X_train)
    #Realizamos la transformación de test
    print(f'[INFO] Realizando transformacion de tipo {type(self.vectorizer).__name__}...')
    X_test_vec = self.vectorizer.transform(X_test)
    return X_train_vec,X_test_vec

  def selectKBestChi(self, X_train: pd.Series, X_test: pd.Series, y_train: pd.Series, y_test: pd.Series) -> tuple:
    """funcion que  realiza la selección de las mejores características del conjunto de train
    : param X_train: pd.Series con los datos de train
    : param X_test: pd.Series con los datos de test
    : param y_train: pd.Series con las etiquetas de train
    : param y_test: pd.Series con las etiquetas de test
    : return: tupla con las series transformadas X_train_best, X_test_best"""
    #Realizamos el entrenamiento y la extracción de train
    print(f'[INFO] Realizando entrenamiento {type(self.selector).__name__}...')
    X_train_best = self.selector.fit_transform(X_train, y_train)
    #Realizamos la transformación de test
    print(f'[INFO] Realizando transformacion de tipo {type(self.vectorizer).__name__}...')
    X_test_best = self.selector.transform(X_test)
    return X_train_best, X_test_best

In [27]:
fe = FeatureExtraction()

# Utilizamos el método run para obtener los mejores resultados
resultados = fe.run(X_train, X_test, y_train, y_test)

[INFO] Realizando entrenamiento TfidfVectorizer...
[INFO] Realizando transformacion de tipo TfidfVectorizer...
[INFO] Realizando entrenamiento CountVectorizer...
[INFO] Realizando transformacion de tipo CountVectorizer...
[INFO] Realizando entrenamiento SelectKBest...
[INFO] Realizando transformacion de tipo CountVectorizer...
[INFO] Realizando entrenamiento SelectKBest...
[INFO] Realizando transformacion de tipo CountVectorizer...


In [28]:
resultados

{'X_train_tdif_best': <4765x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 3758 stored elements in Compressed Sparse Row format>,
 'X_train_vec_best': <4765x10 sparse matrix of type '<class 'numpy.int64'>'
 	with 5031 stored elements in Compressed Sparse Row format>,
 'X_test_tdif_best': <4765x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 3758 stored elements in Compressed Sparse Row format>,
 'X_test_vec_best': <4765x10 sparse matrix of type '<class 'numpy.int64'>'
 	with 5031 stored elements in Compressed Sparse Row format>}

In [ ]:
# CountVectorizer simple
#cv_simple = CountVectorizer()
#X_train_cv_simple = cv_simple.fit_transform(df_train['review_text'])
#X_test_cv_simple = cv_simple.transform(df_test['review_text'])

# CountVectorizer con ngrams, max_features, min_df y max_df
cv_complex = CountVectorizer(ngram_range=(1, 2), max_features=1000, max_df=0.95, min_df=5)
X_train_cv_complex = cv_complex.fit_transform(df_train['review_text']).toarray() #.toarray() para plotearlo
X_test_cv_complex = cv_complex.transform(df_test['review_text']).toarray()

# TfIdfVectorizer simple
#tfidf_simple = TfidfVectorizer()
#X_train_tfidf_simple = tfidf_simple.fit_transform(df_train['review_text'])
#X_test_tfidf_simple = tfidf_simple.transform(df_test['review_text'])

# TfIdfVectorizer complejo
tfidf_complex = TfidfVectorizer(ngram_range=(1, 2), max_features=1000, max_df=0.95, min_df=5)
X_train_tfidf_complex = tfidf_complex.fit_transform(df_train['review_text'])
X_test_tfidf_complex = tfidf_complex.transform(df_test['review_text'])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
#lr_cv_simple = LogisticRegression()
lr_cv_complex = LogisticRegression()
#lr_tfidf_simple = LogisticRegression()
lr_tfidf_complex = LogisticRegression()

In [ ]:
#lr_cv_simple.fit(X_train_cv_simple, df_train['sentiment_label'])  # train
#y_pred_cv_simple = lr_cv_simple.predict(X_test_cv_simple)  # test

In [ ]:
lr_cv_complex.fit(X_train_cv_complex, df_train['sentiment_label'])  # train
y_pred_cv_complex = lr_cv_complex.predict(X_test_cv_complex)  # test

In [ ]:
#lr_tfidf_simple.fit(X_train_tfidf_simple, df_train['sentiment_label'])  # train
#y_pred_tfidf_simple = lr_tfidf_simple.predict(X_test_tfidf_simple)  # test

In [ ]:
lr_tfidf_complex.fit(X_train_tfidf_complex, df_train['sentiment_label'])  # train
y_pred_tfidf_complex = lr_tfidf_complex.predict(X_test_tfidf_complex)  # test

In [ ]:
#print('CountVectorizer simple\n')
#print(confusion_matrix(df_test['sentiment_label'], y_pred_cv_simple))
#print(classification_report(df_test['sentiment_label'], y_pred_cv_simple))

In [ ]:
print('CountVectorizer complejo\n')
print(confusion_matrix(df_test['sentiment_label'], y_pred_cv_complex))
print(classification_report(df_test['sentiment_label'], y_pred_cv_complex))

In [ ]:
#print('TfIdfVectorizer simple\n')
#print(confusion_matrix(df_test['sentiment_label'], y_pred_tfidf_simple))
#print(classification_report(df_test['sentiment_label'], y_pred_tfidf_simple))

In [ ]:
print('TfIdfVectorizer complejo\n')
print(confusion_matrix(df_test['sentiment_label'], y_pred_tfidf_complex))
print(classification_report(df_test['sentiment_label'], y_pred_tfidf_complex))

In [ ]:
#Frecuencia de las palabras. Observamos algunos outliers. 
#Esos datos tienen q ser significativos dado que se ha realizado un preprocesamiento previo
plt.plot(X_train_cv_complex[5,:]) #CountVectorized train en array
plt.xlabel('Word')
plt.ylabel('Count')
plt.show()

In [ ]:

#Observamos la ley de Zipf
word_freq = X_train_cv_complex.sum(axis=0)

sorted_word_freq = np.sort(word_freq)[::-1]

plt.plot(sorted_word_freq)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.xlabel('Rank')
plt.ylabel('Number of occurrences')
plt.show()

In [ ]:
import sklearn.preprocessing as pr
###NORMALIZAAAAAAA!!!!

features_train = pr.normalize(X_train_cv_complex, axis=1)
features_test = pr.normalize(X_test_cv_complex, axis=1)

In [ ]:
#ESTA PARTE ES MOSTRAR DATOS SOLO
cv = TfidfVectorizer(
    max_df=0.95,
    min_df=3,
    max_features=2500,
    strip_accents='ascii',
    ngram_range=(1, 3)
)
cv.fit(X_train)

In [ ]:
print(list(cv.vocabulary_.items())[:20])

In [ ]:
print(len(cv.vocabulary_))

## TF-IDF scores del training set y test set

In [ ]:
#SI YA HE NORMALIZADO ANTES, normalizamos OTRA VEZ??? los conjutos de entrenamiento y test
X_train_ = cv.transform(X_train)
X_test_ = cv.transform(X_test)

##Score IDF de algunas palabras

In [ ]:
words_example = [
    'sauce',
    'very',
    'good',
    'amazing',
    'great',
    'fresh',
    'fish',
    'deviled',
    'but',
    'was',
    'never',
    'not',
    'wasn´t'
]

In [ ]:
vocab_idf = dict(zip(cv.get_feature_names(), cv.idf_))

print('{0:20}{1:20}'.format('Palabra', 'IDF'))
for word in words_example:
    if word not in vocab_idf:
        print('{0:20}{1:20}'.format(word, 'OOV'))
    else:
        print('{0:20}{1:2.3f}'.format(word, vocab_idf[word]))

## Palabras con el TF-IDF en alguna review

In [ ]:
i = random.randint(0, len(X_train))
print('ID: {}'.format(i))
print('Sentiment: {}'.format(y_train.iloc[i]))
print('Review: {}'.format(X_train.iloc[i]))

In [ ]:
doc_vector = X_train_[i]
df_tfidf = pd.DataFrame(doc_vector.T.todense(), index=cv.get_feature_names(), columns=['tfidf'])
df_tfidf = df_tfidf[df_tfidf['tfidf'] > 0]

top_n = 10
print('Top {} words with highest TF_IDF in the review {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[:top_n]))
print('\nTop {} words with lowest TF_IDF in the review {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[-top_n:]))

## χ²  scores

In [ ]:
# Compute the chi-squared score for each word in the training set and show the values
i = 15

chi2score = chi2(X_train_, y_train)[0]
scores = list(zip(cv.get_feature_names(), chi2score))
sorted_scores = sorted(scores, key=lambda x:x[1])
topchi2 = list(zip(*sorted_scores[-i:]))
x = range(len(topchi2[1]))
labels = topchi2[0]

plt.figure(figsize=(12, 8))
plt.barh(x,topchi2[1], align='center', alpha=0.5)
plt.plot(topchi2[1], x, '-o', markersize=5, alpha=0.8)
plt.yticks(x, labels, fontsize=12)
plt.xlabel('$\chi^2$', fontsize=26)
plt.ylabel('word', fontsize=16)
plt.title('Top {} $\chi^2$ score for each word in the training set'.format(i), fontsize=20)
plt.grid()
plt.tight_layout()
plt.show();

## Entrenamiento

In [ ]:
#Hacemos una regresión logistica
c_params = [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000]

train_acc = list()
test_acc = list()
for c in c_params:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(X_train_, y_train)
    
    train_predict = lr.predict(X_train_)
    test_predict = lr.predict(X_test_)
    
    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_test, test_predict)))
    
    train_acc.append(accuracy_score(y_train, train_predict))
    test_acc.append(accuracy_score(y_test, test_predict))

In [ ]:
#imprimimos las métricas
print('Confussion matrix:\n{}'.format(confusion_matrix(y_test, test_predict)))
print('\nClassification report:\n{}'.format(classification_report(y_test, test_predict)))
print('Accuracy score:{}'.format(accuracy_score(y_test, test_predict)))

In [ ]:
#ploteamos
plt.figure(figsize=(12, 8))
plt.plot(train_acc, label='train')
plt.plot(test_acc, label='test')
plt.axvline(np.argmax(test_acc), c='g', ls='--', alpha=0.8)
plt.title('Accuracy evolution for different C values')
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.xticks(list(range(len(c_params))), c_params)
plt.tight_layout()
plt.show()

In [ ]:
p, r, thresholds = precision_recall_curve(y_test, test_predict)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(8, 8))
    plt.title("Precision and Recall Scores as a function of the decision threshold")
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.ylabel("Score")
    plt.xlabel("Decision Threshold")
    plt.legend(loc='best')

In [ ]:
plot_precision_recall_vs_threshold(p, r, thresholds)

##Predicción

In [ ]:
def predict_review_sentiment(review_index, model):
    print('Actual sentiment: {}'.format(df_train.iloc[review_index]['sentiment_label']))
    r = df_train.iloc[review_index]['review_text']
    print('Prediction: {}'.format(lr.predict(cv.transform([r]))))

In [ ]:
for i in random.sample(range(0, len(df_train)), 5):
    print('\nReview no. {}'.format(i))
    predict_review_sentiment(i, lr)

#Modelo nuevo

In [ ]:
#xgboost 
from sklearn.ensemble import GradientBoostingClassifier
labels_train= df_train['sentiment_label']
labels_test= df_test['sentiment_label']
n_estimators = 32

def classify_gboost(X_train, X_test, y_train, y_test):        
    clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=1.0, max_depth=1, random_state=42)

    clf.fit(X_train, y_train)
    
    print("[{}] Accuracy: train = {}, test = {}".format(
            clf.__class__.__name__,
            clf.score(X_train, y_train),
            clf.score(X_test, y_test)))
    
    return clf


clf2 = classify_gboost(features_train, features_test, labels_train, labels_test)